# Data Preparation 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats.mstats import mode

%matplotlib inline
from matplotlib.pyplot import rcParams
rcParams["figure.figsize"] = 12 , 4

In [2]:
train = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/train.csv")
test = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/test.csv")
event_type = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/event_type.csv")
log_feature = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/log_feature.csv")
resource_type = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/resource_type.csv")
severity_type = pd.read_csv("C:/Users/zhanghui/Desktop/Telstra/severity_type.csv")

#### we have four types of features, let's deal with them step by step:

In [3]:
event_type["id"].unique()

array([6597, 8011, 2597, ..., 6488,  878, 4464], dtype=int64)

In [4]:
log_feature["id"].unique()

array([6597, 8011, 2597, ..., 6488,  878, 4464], dtype=int64)

In [5]:
resource_type["id"].unique()

array([6597, 8011, 2597, ..., 6488,  878, 4464], dtype=int64)

In [6]:
severity_type["id"].unique()

array([6597, 8011, 2597, ..., 6488,  878, 4464], dtype=int64)

In [7]:
train["id"].unique()

array([14121,  9320, 14394, ..., 14111, 15189, 17067], dtype=int64)

In [14]:
print (train.shape)
print (test.shape)
print (data.shape)
print (event_type.shape)
print (log_feature.shape)
print (resource_type.shape)
print (severity_type.shape)

(7381, 4)
(11171, 3)
(18552, 4)
(31170, 2)
(58671, 3)
(21076, 2)
(18552, 2)


#### Merge training and test data:

In [10]:
train["source"] = "train"
test["source"] = "test"
data = pd.concat([train , test] , ignore_index=True)

In [12]:
data.head()

,fault_severity,id,location,source
0,1.0,14121,location 118,train
1,0.0,9320,location 91,train
2,1.0,14394,location 152,train
3,1.0,8218,location 931,train
4,0.0,14804,location 120,train


## event type:

In [15]:
event_type["event_type"].unique()
len(event_type["id"].unique())

18552

In [16]:
event_type = event_type.merge(data , on="id")

In [17]:
event_type.head()

,id,event_type,fault_severity,location,source
0,6597,event_type 11,NaN,location 1,test
1,8011,event_type 15,0.0,location 1,train
2,2597,event_type 15,NaN,location 1,test
3,5022,event_type 15,NaN,location 1,test
4,5022,event_type 11,NaN,location 1,test


In [18]:
event_type_unq = pd.DataFrame(event_type["event_type"].value_counts())

In [23]:
pivot_test = event_type.pivot_table(values = "source" , index = "event_type" , aggfunc= lambda x: sum(x == "train"))
pivot_test.head()

,source
event_type,
event_type 1,1
event_type 10,58
event_type 11,3068
event_type 12,2
event_type 13,247


In [20]:
# Determine % of training samples in certain event_type
event_type_unq["PercTrain"] = event_type.pivot_table(values = "source" , index = "event_type" , aggfunc = lambda x: sum(x == "train") / float(len(x)))
event_type_unq.head()

,event_type,PercTrain
event_type 11,7888,0.388945
event_type 35,6615,0.407105
event_type 34,5927,0.406783
event_type 15,4395,0.392264
event_type 20,1458,0.382030


In [21]:
pivot_test2 = event_type.loc[event_type["source"] == "train"].pivot_table(values = "fault_severity", index = "event_type" , aggfunc = mode)
pivot_test2.head()

,fault_severity
event_type,
event_type 1,"([0.0], [1.0])"
event_type 10,"([1.0], [30.0])"
event_type 11,"([0.0], [1677.0])"
event_type 12,"([0.0], [1.0])"
event_type 13,"([1.0], [136.0])"


In [22]:
event_type_unq["Mode_Severity"] = event_type.loc[event_type["source"] == "train"].pivot_table(values = "fault_severity" , index = "event_type" , aggfunc = lambda x:mode(x).mode[0])

In [24]:
event_type_unq["preprocess"] = event_type_unq.index.values

In [25]:
event_type_unq.head()

,event_type,PercTrain,Mode_Severity,preprocess
event_type 11,7888,0.388945,0.0,event_type 11
event_type 35,6615,0.407105,0.0,event_type 35
event_type 34,5927,0.406783,0.0,event_type 34
event_type 15,4395,0.392264,0.0,event_type 15
event_type 20,1458,0.382030,0.0,event_type 20


In [26]:
top_unchange = 33
event_type_unq["preprocess"].iloc[top_unchange:] = event_type_unq["Mode_Severity"].iloc[top_unchange:].apply(lambda x: "Remove" if pd.isnull(x) else "event_type others_%d"%int(x))

C:\Users\zhanghui\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
event_type_unq["preprocess"].value_counts()

event_type others_0    10
event_type others_2     5
Remove                  4
event_type 15           1
event_type 13           1
event_type 35           1
event_type 14           1
event_type 20           1
event_type 40           1
event_type 54           1
event_type 43           1
event_type 46           1
event_type 50           1
event_type 26           1
event_type 29           1
event_type 11           1
event_type 42           1
event_type 22           1
event_type 45           1
event_type 7            1
event_type 32           1
event_type 18           1
event_type 47           1
event_type 23           1
event_type 28           1
event_type 44           1
event_type 27           1
event_type 21           1
event_type 8            1
event_type 10           1
event_type others_1     1
event_type 5            1
event_type 6            1
event_type 30           1
event_type 2            1
event_type 24           1
event_type 34           1
Name: preprocess, dtype: int64

In [28]:
event_type.head()

,id,event_type,fault_severity,location,source
0,6597,event_type 11,NaN,location 1,test
1,8011,event_type 15,0.0,location 1,train
2,2597,event_type 15,NaN,location 1,test
3,5022,event_type 15,NaN,location 1,test
4,5022,event_type 11,NaN,location 1,test


In [29]:
event_type = event_type.merge(event_type_unq[["preprocess"]] , left_on="event_type" , right_index=True)

In [30]:
event_type.head()

,id,event_type,fault_severity,location,source,preprocess
0,6597,event_type 11,NaN,location 1,test,event_type 11
4,5022,event_type 11,NaN,location 1,test,event_type 11
5,6852,event_type 11,NaN,location 1,test,event_type 11
9,14838,event_type 11,NaN,location 1,test,event_type 11
11,2588,event_type 11,0.0,location 1,train,event_type 11


In [32]:
event_type_merge = event_type.pivot_table(values="event_type" , index = "id" , columns="preprocess" , aggfunc=lambda x: len(x) , fill_value=0)

In [33]:
event_type_merge.shape

(18552, 37)

In [34]:
event_type.pivot_table(values="event_type" , index = "id" , aggfunc=lambda x:len(x))

,event_type
id,
1,2
2,2
3,1
4,1
5,2
6,1
7,2
8,2
9,2


In [35]:
data = data.merge(event_type_merge , left_on="id" , right_index=True)

In [36]:
data.head()

,fault_severity,id,location,source,Remove,event_type 10,event_type 11,event_type 13,event_type 14,event_type 15,...,event_type 47,event_type 5,event_type 50,event_type 54,event_type 6,event_type 7,event_type 8,event_type others_0,event_type others_1,event_type others_2
0,1.0,14121,location 118,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,9320,location 91,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,14394,location 152,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,8218,location 931,train,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.0,14804,location 120,train,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [37]:
data.shape

(18552, 41)

## log feature:

In [38]:
log_feature.head()

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1
3,5022,feature 172,2
4,5022,feature 56,1


In [39]:
log_feature["log_feature"].value_counts()

feature 312    5267
feature 232    4754
feature 82     3472
feature 203    2823
feature 313    2145
feature 233    1901
feature 307    1597
feature 54     1573
feature 170    1526
feature 71     1514
feature 315    1495
feature 134    1419
feature 80     1336
feature 235    1294
feature 193    1160
feature 219    1152
feature 68     1093
feature 227    1080
feature 314     950
feature 201     902
feature 234     882
feature 73      868
feature 195     783
feature 301     707
feature 309     627
feature 55      564
feature 229     560
feature 273     491
feature 308     484
feature 368     462
               ... 
feature 333       1
feature 17        1
feature 372       1
feature 334       1
feature 214       1
feature 329       1
feature 355       1
feature 123       1
feature 297       1
feature 208       1
feature 269       1
feature 364       1
feature 300       1
feature 175       1
feature 119       1
feature 263       1
feature 326       1
feature 13        1
feature 12        1


In [40]:
log_feature.shape

(58671, 3)

In [41]:
log_feature = log_feature.merge(data[["id" , "fault_severity" , "source"]] , on = "id")

In [42]:
log_feature.head()

,id,log_feature,volume,fault_severity,source
0,6597,feature 68,6,NaN,test
1,8011,feature 68,7,0.0,train
2,2597,feature 68,1,NaN,test
3,5022,feature 172,2,NaN,test
4,5022,feature 56,1,NaN,test


In [43]:
log_feature_unq = pd.DataFrame(log_feature["log_feature"].value_counts())
log_feature_unq.head()

,log_feature
feature 312,5267
feature 232,4754
feature 82,3472
feature 203,2823
feature 313,2145


In [44]:
log_feature_unq["PercTrain"] = log_feature.pivot_table(values="source" , index = "log_feature" , aggfunc=lambda x: sum(x == "train")/float(len(x)))
log_feature_unq.head()

,log_feature,PercTrain
feature 312,5267,0.411430
feature 232,4754,0.408919
feature 82,3472,0.393433
feature 203,2823,0.393199
feature 313,2145,0.421445


In [45]:
log_feature_unq["Mode_Severity"] = log_feature.loc[log_feature["source"] == "train"].pivot_table(values="fault_severity" , index = "log_feature" , aggfunc=lambda x: mode(x).mode[0])
log_feature_unq.head()

,log_feature,PercTrain,Mode_Severity
feature 312,5267,0.411430,0.0
feature 232,4754,0.408919,0.0
feature 82,3472,0.393433,2.0
feature 203,2823,0.393199,2.0
feature 313,2145,0.421445,0.0


In [46]:
log_feature_unq["preprocess"] = log_feature_unq.index.values
log_feature_unq["preprocess"].loc[log_feature_unq["PercTrain"] == 1] = np.nan

C:\Users\zhanghui\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
top_unchange = 128
log_feature_unq["preprocess"].iloc[top_unchange:] = log_feature_unq["Mode_Severity"].iloc[top_unchange:].apply(lambda x: "Remove" if pd.isnull(x) else "feature others_%d"%int(x))

C:\Users\zhanghui\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [48]:
log_feature_unq["preprocess"].value_counts()

feature others_0    111
feature others_1     64
Remove               55
feature others_2     28
feature 318           1
feature 135           1
feature 68            1
feature 105           1
feature 103           1
feature 240           1
feature 85            1
feature 234           1
feature 309           1
feature 313           1
feature 284           1
feature 101           1
feature 229           1
feature 220           1
feature 179           1
feature 95            1
feature 307           1
feature 191           1
feature 87            1
feature 203           1
feature 170           1
feature 94            1
feature 198           1
feature 273           1
feature 66            1
feature 62            1
                   ... 
feature 153           1
feature 73            1
feature 291           1
feature 306           1
feature 132           1
feature 345           1
feature 20            1
feature 118           1
feature 154           1
feature 211           1
feature 76      

In [49]:
log_feature.head()

,id,log_feature,volume,fault_severity,source
0,6597,feature 68,6,NaN,test
1,8011,feature 68,7,0.0,train
2,2597,feature 68,1,NaN,test
3,5022,feature 172,2,NaN,test
4,5022,feature 56,1,NaN,test


In [50]:
log_feature = log_feature.merge(log_feature_unq[["preprocess"]] , left_on="log_feature" , right_index=True)

In [51]:
log_feature_merge = log_feature.pivot_table(values="volume" ,index = "id" , columns="preprocess" , aggfunc=np.sum ,fill_value=0)

In [52]:
log_feature_merge.head()

preprocess,Remove,feature 101,feature 103,feature 105,feature 109,feature 118,feature 132,feature 133,feature 134,feature 135,...,feature 82,feature 83,feature 85,feature 86,feature 87,feature 94,feature 95,feature others_0,feature others_1,feature others_2
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
log_feature_merge.sum()

preprocess
Remove                277
feature 101          4643
feature 103          5900
feature 105          2265
feature 109          3880
feature 118            43
feature 132            96
feature 133           117
feature 134          1885
feature 135           150
feature 153           166
feature 154           101
feature 155           391
feature 157            80
feature 160           359
feature 161            36
feature 163           291
feature 167           300
feature 170         11697
feature 171          2154
feature 172          4451
feature 179           793
feature 181           122
feature 182           166
feature 188            85
feature 191          1288
feature 193          5572
feature 195          6697
feature 196           636
feature 197           235
                    ...  
feature 44            334
feature 47            247
feature 51           8665
feature 52            362
feature 54          22217
feature 55           2406
feature 56           8829
f

In [54]:
log_feature_merge.sum().sum()

568246

In [55]:
log_feature_merge.shape

(18552, 132)

In [56]:
data = data.merge(log_feature_merge , left_on="id" , right_index=True)

In [57]:
data.head()

,fault_severity,id,location,source,Remove_x,event_type 10,event_type 11,event_type 13,event_type 14,event_type 15,...,feature 82,feature 83,feature 85,feature 86,feature 87,feature 94,feature 95,feature others_0,feature others_1,feature others_2
0,1.0,14121,location 118,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,9320,location 91,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,14394,location 152,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,8218,location 931,train,0,0,1,0,0,1,...,12,0,0,0,0,0,0,0,0,0
4,0.0,14804,location 120,train,0,0,1,0,0,0,...,0,0,0,0,0,0,0,4,0,0


## Resource Type:

In [58]:
resource_type.head()

,id,resource_type
0,6597,resource_type 8
1,8011,resource_type 8
2,2597,resource_type 8
3,5022,resource_type 8
4,6852,resource_type 8


In [59]:
resource_type["resource_type"].value_counts()

resource_type 8     10268
resource_type 2      8918
resource_type 6       582
resource_type 7       498
resource_type 4       330
resource_type 9       190
resource_type 3       145
resource_type 10       73
resource_type 1        58
resource_type 5        14
Name: resource_type, dtype: int64

In [60]:
resource_type.shape

(21076, 2)

In [61]:
resource_type = resource_type.merge(data[["id" , "fault_severity" , "source"]] , on = "id")

In [62]:
resource_type_unq = pd.DataFrame(resource_type["resource_type"].value_counts())

In [63]:
resource_type_unq["PercTrain"] = resource_type.pivot_table(values = "source" , index = "resource_type" , aggfunc=lambda x: sum(x == "train")/float(len(x)))

In [64]:
resource_type_unq.head()

,resource_type,PercTrain
resource_type 8,10268,0.394527
resource_type 2,8918,0.401996
resource_type 6,582,0.424399
resource_type 7,498,0.451807
resource_type 4,330,0.436364


In [65]:
resource_type_unq["Mode_Severity"] = resource_type.loc[resource_type["source"] == "train"].pivot_table(values="fault_severity" , index="resource_type" , aggfunc=lambda x:mode(x).mode[0])
resource_type_unq.head()

,resource_type,PercTrain,Mode_Severity
resource_type 8,10268,0.394527,0.0
resource_type 2,8918,0.401996,0.0
resource_type 6,582,0.424399,1.0
resource_type 7,498,0.451807,0.0
resource_type 4,330,0.436364,0.0


In [66]:
resource_type.loc[resource_type["resource_type"] == "resource_type 5"]

,id,resource_type,fault_severity,source
5653,5475,resource_type 5,NaN,test
5655,5915,resource_type 5,NaN,test
5657,9989,resource_type 5,NaN,test
6936,7378,resource_type 5,2.0,train
7064,9373,resource_type 5,2.0,train
7073,9677,resource_type 5,NaN,test
7223,6322,resource_type 5,NaN,test
10786,15655,resource_type 5,NaN,test
10790,11683,resource_type 5,NaN,test
10793,1616,resource_type 5,2.0,train


In [67]:
resource_type_merge = resource_type.pivot_table(values="source" , index = "id" , columns="resource_type" , aggfunc=lambda x:len(x), fill_value=0)

In [68]:
data = data.merge(resource_type_merge , left_on="id" , right_index=True)

In [69]:
data.head()

,fault_severity,id,location,source,Remove_x,event_type 10,event_type 11,event_type 13,event_type 14,event_type 15,...,resource_type 1,resource_type 10,resource_type 2,resource_type 3,resource_type 4,resource_type 5,resource_type 6,resource_type 7,resource_type 8,resource_type 9
0,1.0,14121,location 118,train,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0.0,9320,location 91,train,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1.0,14394,location 152,train,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1.0,8218,location 931,train,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0.0,14804,location 120,train,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [70]:
data.shape

(18552, 183)

## Severity type:

In [71]:
severity_type.head()

,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1


In [72]:
severity_type.shape

(18552, 2)

In [73]:
severity_type = severity_type.merge(data[["id" , "fault_severity" , "source"]], on = "id")
severity_type.head()

,id,severity_type,fault_severity,source
0,6597,severity_type 2,NaN,test
1,8011,severity_type 2,0.0,train
2,2597,severity_type 2,NaN,test
3,5022,severity_type 1,NaN,test
4,6852,severity_type 1,NaN,test


In [74]:
severity_type_unq = pd.DataFrame(severity_type["severity_type"].value_counts())
severity_type_unq.head()

,severity_type
severity_type 2,8737
severity_type 1,8728
severity_type 4,1014
severity_type 5,65
severity_type 3,8


In [75]:
severity_type_unq["PercTrain"] = severity_type.pivot_table(values="source" , index = "severity_type" , aggfunc=lambda x:sum(x == "train")/float(len(x)))
severity_type_unq.head()

,severity_type,PercTrain
severity_type 2,8737,0.411011
severity_type 1,8728,0.386687
severity_type 4,1014,0.382643
severity_type 5,65,0.353846
severity_type 3,8,0.500000


In [76]:
severity_type_unq["Mode_Severity"] = severity_type.loc[severity_type["source"] == "train"].pivot_table(values="fault_severity" , index="severity_type" , aggfunc=lambda x:mode(x).mode[0])
severity_type_unq.head()

,severity_type,PercTrain,Mode_Severity
severity_type 2,8737,0.411011,0.0
severity_type 1,8728,0.386687,0.0
severity_type 4,1014,0.382643,0.0
severity_type 5,65,0.353846,0.0
severity_type 3,8,0.500000,0.0


In [77]:
severity_type.loc[severity_type["source"] == "train"].pivot_table(values="fault_severity" , index="severity_type" , aggfunc=lambda x:mode(x))

,fault_severity
severity_type,
severity_type 1,"([0.0], [1778.0])"
severity_type 2,"([0.0], [2652.0])"
severity_type 3,"([0.0], [4.0])"
severity_type 4,"([0.0], [338.0])"
severity_type 5,"([0.0], [12.0])"


In [78]:
severity_type_unq["Mode_Severity"].value_counts()

0.0    5
Name: Mode_Severity, dtype: int64

In [79]:
severity_type_unq.shape

(5, 3)

In [80]:
severity_type_merge = severity_type.pivot_table(values="source" , index="id" , columns="severity_type" , aggfunc=lambda x:len(x) , fill_value=0)


In [81]:
data = data.merge(severity_type_merge , left_on="id" , right_index=True)

In [82]:
data.head()

,fault_severity,id,location,source,Remove_x,event_type 10,event_type 11,event_type 13,event_type 14,event_type 15,...,resource_type 5,resource_type 6,resource_type 7,resource_type 8,resource_type 9,severity_type 1,severity_type 2,severity_type 3,severity_type 4,severity_type 5
0,1.0,14121,location 118,train,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0.0,9320,location 91,train,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1.0,14394,location 152,train,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1.0,8218,location 931,train,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,0,0
4,0.0,14804,location 120,train,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [83]:
data.shape

(18552, 188)

In [84]:
pred_event = [x for x in data.columns if "event_type" in x]

In [85]:
data[pred_event].sum().sum()

31164

In [86]:
pred_feat = [x for x in data.columns if "feature" in x]

In [87]:
data[pred_feat].sum().sum()

567969

In [88]:
pred_res = [x for x in data.columns if "resource" in x]

In [89]:
data[pred_res].sum().sum()

21076

In [90]:
pred_sev = [x for x in data.columns if "severity_type" in x]

In [91]:
data[pred_sev].sum().sum()

18552

## Add count variables:

In [92]:
location_counts = data["location"].value_counts()
data["location_counts"] = data["location"].apply(lambda x:location_counts[x])

In [93]:
len(location_counts)

1126

In [94]:
featvar = [x for x in data.columns if "feature " in x]

In [95]:
data["feature_count"] = data[featvar].apply(np.sum , axis = 1)
data["feature_count"].sum()

567969

In [96]:
data[["location" , "location_counts"]]

,location,location_counts
0,location 118,97
1,location 91,98
2,location 152,12
3,location 931,69
4,location 120,19
5,location 664,11
6,location 640,11
7,location 122,134
8,location 263,5
9,location 613,110


In [97]:
le = LabelEncoder()

In [98]:
data["location"] = le.fit_transform(data["location"])

In [99]:
[x for x in data.columns if "Remove" in x]

['Remove_x', 'Remove_y']

In [100]:
data.drop(["Remove_x" , "Remove_y"] , axis = 1 , inplace = True)

In [101]:
train_mod = data.loc[data["source"] == "train"]
test_mod = data.loc[data["source"] == "test"]

In [102]:
train_mod.drop("source" , axis = 1 , inplace=True)
test_mod.drop(["source" , "fault_severity"] , axis = 1 , inplace = True)

C:\Users\zhanghui\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\zhanghui\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [104]:
train_mod.to_csv("C:/Users/zhanghui/Desktop/Telstra/train_modified_1.csv" , index = False)
test_mod.to_csv("C:/Users/zhanghui/Desktop/Telstra/test_modified_1.csv" , index = False)